In [1]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import KFold

train = pd.read_csv('../input/titanic/train.csv')

In [2]:
kf = KFold(n_splits=3, random_state=1001,shuffle=True)
for i, (train_index, val_index) in enumerate(kf.split(train)):
    trn= train.iloc[train_index].reset_index()
    val= train.iloc[val_index].reset_index()
    
trn = trn.drop(columns=['index'])
val = val.drop(columns=['index'])

val.to_csv('sub_val.csv',index=False)
trn.to_csv('sub_train.csv',index=False)

In [3]:
# !rm -r ./autox
!git clone https://github.com/4paradigm/autox.git
!pip install ./autox

Cloning into 'autox'...
remote: Enumerating objects: 2699, done.
remote: Counting objects: 100% (612/612), done.
remote: Compressing objects: 100% (153/153), done.
remote: Total 2699 (delta 479), reused 570 (delta 452), pack-reused 2087
Receiving objects: 100% (2699/2699), 11.87 MiB | 18.85 MiB/s, done.
Resolving deltas: 100% (1723/1723), done.
Processing ./autox
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.3 MB/s eta 0:00:0000:0100:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.9/948.9 KB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.4/325.4 KB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 33.8 MB/s eta 0:00:0000:0100:01
  Created wheel for automl-x: filename=automl_x

In [4]:
from autox.autox_nlp import NLP_feature
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [5]:
df_train = pd.read_csv('sub_train.csv')
df_test = pd.read_csv('sub_val.csv')

In [6]:
use_Toknizer=True
emb_mode = 'Bert'# TFIDF / Word2Vec / Glove / FastText / Bert
encode_mode = 'supervise' # unsupervise / supervise
text_columns_name = ['Name']
target_column = df_train['Survived']
candidate_labels=None

In [7]:
nlp = NLP_feature()
# nlp.do_mlm = True
# nlp.mlm_epochs=3
# nlp.model_name = 'microsoft/deberta-v3-base'
nlp.emb_size=100
nlp.n_clusters=20
df = nlp.fit(df_train,
             text_columns_name,
             use_Toknizer,
             emb_mode,
             encode_mode,
             target_column,
             candidate_labels)

Fitting column: Name tokenizer


Downloading:   0%|          | 0.00/285 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Fitting column: Name bert embedding


Downloading:   0%|          | 0.00/16.9M [00:00<?, ?B/s]

Some weights of the model checkpoint at prajjwal1/bert-tiny were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Saving model checkpoint to ./Name_transformer
Configuration saved in ./Name_transformer/config.json
Model weights saved in ./Name_transformer/pytorch_model.bin
loading configuration file ./Name_transformer/config.json
Model config BertConfig {
  "_name_or_path": "./Name_transformer",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob"

Fitting column: Name encoder


loading configuration file ./Name_transformer/config.json
Model config BertConfig {
  "_name_or_path": "./Name_transformer",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file ./Name_transformer/pytorch_model.bin
Some weights of the model checkpoint at ./Name_transformer were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.

In [8]:
for column in df.columns:
    df_train[column] = df[column]
df_train = df_train.drop(columns=text_columns_name)

In [9]:
test = nlp.transform(df_test)

loading configuration file ./Name_transformer/config.json
Model config BertConfig {
  "_name_or_path": "./Name_transformer",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 128,
  "initializer_range": 0.02,
  "intermediate_size": 512,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 2,
  "num_hidden_layers": 2,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file ./Name_transformer/pytorch_model.bin
Some weights of the model checkpoint at ./Name_transformer were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.

Transforming column: Name


In [10]:
for column in test.columns:
    df_test[column] = test[column]
df_test = df_test.drop(columns=text_columns_name)

In [11]:
df_train.to_csv(f'{emb_mode}_{encode_mode}_autox_trn.csv',index=False)
df_test.to_csv(f'{emb_mode}_{encode_mode}_autox_val.csv',index=False)

In [13]:
df_val=pd.read_csv(f'{emb_mode}_{encode_mode}_autox_val.csv').drop(columns=['Survived'])
df_val.to_csv(f'{emb_mode}_{encode_mode}_autox_tst.csv',index=False)

In [14]:
from autox import AutoX

path = f'.' 
autox = AutoX(target = 'Survived', train_name = f'{emb_mode}_{encode_mode}_autox_trn.csv', test_name = f'{emb_mode}_{encode_mode}_autox_tst.csv',  id = ['PassengerId'], path = path)
sub = autox.get_submit()

   INFO ->  [+] read Bert_supervise_autox_tst.csv
   INFO ->  Memory usage of dataframe is 0.03 MB
   INFO ->  Memory usage after optimization is: 0.02 MB
   INFO ->  Decreased by 30.9%
   INFO ->  table = Bert_supervise_autox_tst.csv, shape = (297, 11)
   INFO ->  [+] read Bert_supervise_autox_trn.csv
   INFO ->  Memory usage of dataframe is 0.05 MB
   INFO ->  Memory usage after optimization is: 0.03 MB
   INFO ->  Decreased by 37.1%
   INFO ->  table = Bert_supervise_autox_trn.csv, shape = (594, 12)
   INFO ->  [+] read Bert_supervise_autox_val.csv
   INFO ->  Memory usage of dataframe is 0.03 MB
   INFO ->  Memory usage after optimization is: 0.02 MB
   INFO ->  Decreased by 35.6%
   INFO ->  table = Bert_supervise_autox_val.csv, shape = (297, 12)
   INFO ->  [+] read sub_val.csv
   INFO ->  Memory usage of dataframe is 0.03 MB
   INFO ->  Memory usage after optimization is: 0.03 MB
   INFO ->  Decreased by -1.7%
   INFO ->  table = sub_val.csv, shape = (297, 12)
   INFO ->  [+] re

Training on fold 1
Training until validation scores don't improve for 150 rounds
[100]	training's auc: 0.929411	valid_1's auc: 0.863327
[200]	training's auc: 0.940415	valid_1's auc: 0.866729
[300]	training's auc: 0.953514	valid_1's auc: 0.865492
Early stopping, best iteration is:
[190]	training's auc: 0.939254	valid_1's auc: 0.867965
AUC: 0.867965367965368
Fold 1 finished in 0:00:00.184559
Training on fold 2
Training until validation scores don't improve for 150 rounds
[100]	training's auc: 0.93161	valid_1's auc: 0.873574
[200]	training's auc: 0.944124	valid_1's auc: 0.866967
Early stopping, best iteration is:
[51]	training's auc: 0.9242	valid_1's auc: 0.877928
AUC: 0.877927927927928
Fold 2 finished in 0:00:00.103879
Training on fold 3
Training until validation scores don't improve for 150 rounds
[100]	training's auc: 0.928303	valid_1's auc: 0.909615
Early stopping, best iteration is:
[4]	training's auc: 0.913476	valid_1's auc: 0.926603
AUC: 0.9266025641025641
Fold 3 finished in 0:00:0

   INFO ->  feature importance
   INFO ->                                feature  fold_1  fold_2  fold_3  fold_4  fold_5  \
0                                 Age     323      80       9     411       6   
1                                Fare     307      84      11     286       9   
2                   Name_meta_feature     208      76      10     368       7   
3            Cabin__PassengerId__rank     190      51       6     332       2   
4                    Cabin__Age__rank     164      59       2     263       3   
..                                ...     ...     ...     ...     ...     ...   
94             Cabin__Parch__shift__2       0       0       0       0       0   
95             Cabin__Parch__shift__3       0       0       0       0       0   
96             Cabin__Fare__shift__-1       0       0       0       0       0   
97              Cabin__Fare__shift__1       0       0       0       0       0   
98  Cabin__Name_meta_feature__diff__3       0       0       0     

[200]	training's auc: 0.946426	valid_1's auc: 0.870085
[300]	training's auc: 0.958449	valid_1's auc: 0.87094
Early stopping, best iteration is:
[227]	training's auc: 0.95035	valid_1's auc: 0.873219
AUC: 0.8732193732193733
Fold 4 finished in 0:00:00.173914
Training on fold 5
Training until validation scores don't improve for 150 rounds
[100]	training's auc: 0.942005	valid_1's auc: 0.796347
Early stopping, best iteration is:
[3]	training's auc: 0.926839	valid_1's auc: 0.812481
AUC: 0.8124809741248098
Fold 5 finished in 0:00:00.098182
Training on fold 1
AUC: 0.8478260869565217
Fold 1 finished in 0:00:02.750917
Training on fold 2
AUC: 0.8061104582843713
Fold 2 finished in 0:00:01.379876
Training on fold 3
AUC: 0.8789682539682541
Fold 3 finished in 0:00:01.457884
Training on fold 4
AUC: 0.7579318448883665
Fold 4 finished in 0:00:01.409821
Training on fold 5
AUC: 0.8518518518518519
Fold 5 finished in 0:00:01.391494
Training on fold 6
AUC: 0.8082706766917294
Fold 6 finished in 0:00:01.356802


   INFO ->  Average KFold AUC: 0.8142876976199765


AUC: 0.7888235294117647
Fold 10 finished in 0:00:01.387209


In [15]:
val = pd.read_csv(f'sub_val.csv')
from sklearn.metrics import mean_squared_error,roc_auc_score
AUC = (roc_auc_score(val['Survived'], sub['Survived']))

In [17]:
AUC

0.8578347578347577